In [1]:
import sys
print(sys.version)



3.9.23 | packaged by conda-forge | (main, Jun  4 2025, 17:57:12) 
[GCC 13.3.0]


In [2]:
!python -m pip install pandas


In [3]:
!python -m pip install pydantic

In [4]:
!python -m pip install pyyaml


In [5]:
!python -m pip install pyarrow


In [6]:
!python -m pip install --upgrade "pyarrow>=17.0.0" "pandas>=2.2.2"


In [7]:
!python -m pip install "numpy<2" torch==2.1.0 torchdata==0.7.0 dgl==2.1.0


In [8]:
!python -m pip show torch dgl torchdata


Name: torch
Version: 2.1.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: torchdata
---
Name: dgl
Version: 2.1.0
Summary: Deep Graph Library
Home-page: https://github.com/dmlc/dgl
Author: 
Author-email: 
License: APACHE
Location: /home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages
Requires: networkx, numpy, psutil, requests, scipy, torchdata, tqdm
Required-by: 
---
Name: torchdata
Version: 0.7.0
Summary: Composable data loading module

In [9]:
import torch, torchdata, dgl
print(torch.__version__)
print(dgl.__version__)
print(torchdata.__version__)


2.1.0+cu121
2.1.0
0.7.0


In [10]:
!pwd
!ls
!ls country_borders



/home/studio-lab-user/sagemaker-studiolab-notebooks/country_borders
country_dgl.ipynb  country_edges_land_sea.csv  country_geo_bg_vectors.parquet
ls: cannot access 'country_borders': No such file or directory


In [11]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
import dgl.data
import dgl.nn as dglnn

In [14]:
import os
import pandas as pd

# Move to the directory that contains "country_borders"
os.chdir("/home/studio-lab-user/sagemaker-studiolab-notebooks")

# Sanity check
print(os.getcwd())
print(os.listdir("."))  # you should see 'country_borders' here

# Now this will work
csv_path = "country_borders/country_edges_land_sea.csv"
edges = pd.read_csv(csv_path)
edges.head()


/home/studio-lab-user/sagemaker-studiolab-notebooks
['Getting Started.ipynb', 'images', '.ipynb_checkpoints', 'testDgl.ipynb', 'country_borders']


,node1,node2,how
0,AFG,CHN,land
1,AFG,IRN,land
2,AFG,PAK,land
3,AFG,TJK,land
4,AFG,TKM,land


In [15]:

import os
import pandas as pd

# Go to the directory that contains "country_borders"
os.chdir("/home/studio-lab-user/sagemaker-studiolab-notebooks")

print(os.getcwd())
print(os.listdir("."))        # you should see 'country_borders' here

pq_path = "country_borders/country_geo_bg_vectors.parquet"
vectors = pd.read_parquet(pq_path)
vectors.head()




/home/studio-lab-user/sagemaker-studiolab-notebooks
['Getting Started.ipynb', 'images', '.ipynb_checkpoints', 'testDgl.ipynb', 'country_borders']


,country,iso3,country_geo_bg_vector
0,Afghanistan,AFG,"[-0.08817123621702194, 0.007525322493165731, -..."
1,Albania,ALB,"[-0.02698056772351265, 0.0006709833978675306, ..."
2,Algeria,DZA,"[-0.04456178843975067, 0.01614188216626644, -0..."
3,American Samoa,ASM,"[0.05785828456282616, -0.04124083369970322, -0..."
4,Andorra,AND,"[0.0006614752346649766, 0.019504521042108536, ..."


In [16]:
nodes=vectors.copy()
nodes = nodes.reset_index(drop=True)
nodes["nid"] = range(len(nodes))

In [17]:
nodes.tail(2)

,country,iso3,country_geo_bg_vector,nid
231,Zambia,ZMB,"[-0.07020933926105499, 0.0007883943035267293, ...",231
232,Zimbabwe,ZWE,"[-0.01679103821516037, 0.018411774188280106, -...",232


In [18]:
iso3_to_id = dict(zip(nodes["iso3"], nodes["nid"]))

In [19]:
# Keep only edges where both endpoints exist in node table
edges = edges[edges["node1"].isin(iso3_to_id) & edges["node2"].isin(iso3_to_id)].copy()

edges["src"] = edges["node1"].map(iso3_to_id)
edges["dst"] = edges["node2"].map(iso3_to_id)

# Make the graph undirected by adding reverse edges (optional but common)
edges_undirected = pd.concat(
    [
        edges[["src", "dst", "how"]],
        edges.rename(columns={"src": "dst", "dst": "src"})[["src", "dst", "how"]],
    ],
    ignore_index=True,
)


In [20]:
edges_undirected.tail(2)

,src,dst,how
970,227,216,sea
971,225,176,sea


In [21]:
import torch
import dgl

src = torch.tensor(edges_undirected["src"].to_numpy(), dtype=torch.int64)
dst = torch.tensor(edges_undirected["dst"].to_numpy(), dtype=torch.int64)

g = dgl.graph((src, dst), num_nodes=len(nodes))
print(g)


Graph(num_nodes=233, num_edges=972,
      ndata_schemes={}
      edata_schemes={})


In [22]:
# node features: list-of-floats → tensor [num_nodes, dim]
feat_tensor = torch.tensor(
    nodes.sort_values("nid")["country_geo_bg_vector"].to_list(),
    dtype=torch.float32,
)

g.ndata["feat"] = feat_tensor


/tmp/ipykernel_93/3608011557.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  feat_tensor = torch.tensor(


In [23]:
g

Graph(num_nodes=233, num_edges=972,
      ndata_schemes={'feat': Scheme(shape=(384,), dtype=torch.float32)}
      edata_schemes={})

In [24]:
u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.131)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [25]:
train_g = dgl.remove_edges(g, eids[:test_size])

In [26]:
train_g, test_size

(Graph(num_nodes=233, num_edges=845,
       ndata_schemes={'feat': Scheme(shape=(384,), dtype=torch.float32)}
       edata_schemes={}),
 127)

In [27]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [28]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [29]:
train_neg_g

Graph(num_nodes=233, num_edges=845,
      ndata_schemes={}
      edata_schemes={})

In [30]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [31]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [32]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 128)
# You can replace DotPredictor with MLPPredictor.
pred = MLPPredictor(128)
# pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [33]:
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.001)

In [34]:
all_logits = []
for e in range(2000):
    # forward
    h = model(train_g, train_g.ndata['feat'].float())
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if e % 100 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

In epoch 0, loss: 0.6947704553604126
In epoch 100, loss: 0.01416592113673687
In epoch 200, loss: 0.005903133191168308
In epoch 300, loss: 0.005387501325458288
In epoch 400, loss: 0.00522635318338871
In epoch 500, loss: 0.005288422107696533
In epoch 600, loss: 0.005127193406224251
In epoch 700, loss: 0.005238433834165335
In epoch 800, loss: 0.005151073914021254
In epoch 900, loss: 0.005083347670733929
In epoch 1000, loss: 0.005121178459376097
In epoch 1100, loss: 0.005258490797132254
In epoch 1200, loss: 0.005100707523524761
In epoch 1300, loss: 0.005083524156361818
In epoch 1400, loss: 0.005217311438173056
In epoch 1500, loss: 0.005065891891717911
In epoch 1600, loss: 0.005054098088294268
In epoch 1700, loss: 0.005065654870122671
In epoch 1800, loss: 0.005046406760811806
In epoch 1900, loss: 0.00505028897896409


In [38]:
!pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 111.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [39]:
from sklearn.metrics import roc_auc_score


In [40]:
# from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))

AUC 0.9689379378758757


In [49]:
h.shape

torch.Size([233, 128])

In [45]:
nodes.tail(2)

,country,iso3,country_geo_bg_vector,nid
231,Zambia,ZMB,"[-0.07020933926105499, 0.0007883943035267293, ...",231
232,Zimbabwe,ZWE,"[-0.01679103821516037, 0.018411774188280106, -...",232


In [46]:
import torch
import numpy as np
import pandas as pd

# 1) Keep only the columns you want
nodes_basic = nodes[["country", "iso3", "nid"]].copy()

# 2) Convert h → numpy
h_np = h.detach().cpu().numpy()   # shape (N, D)

# 3) Attach GNN embedding by nid
nodes_basic["h_vector"] = nodes_basic["nid"].apply(
    lambda i: h_np[int(i)].tolist()
)




In [47]:
nodes_basic.tail(2)

,country,iso3,nid,h_vector
231,Zambia,ZMB,231,"[-4.777525424957275, -1.3836770057678223, -0.1..."
232,Zimbabwe,ZWE,232,"[-4.723304271697998, -1.485762119293213, -0.08..."


In [48]:
# 4) Save (parquet is nice)
nodes_basic.to_parquet("country_nodes_with_h.parquet", index=False)
# or:
# nodes_basic.to_csv("country_nodes_with_h.csv", index=False)

In [50]:
import pandas as pd

df_nodes = pd.read_parquet("country_nodes_with_h.parquet")
df_nodes.head()


,country,iso3,nid,h_vector
0,Afghanistan,AFG,0,"[-1.1849541664123535, 1.1546564102172852, -2.5..."
1,Albania,ALB,1,"[-2.417677640914917, 1.6790118217468262, -1.46..."
2,Algeria,DZA,2,"[-5.533444404602051, 0.032391972839832306, -0...."
3,American Samoa,ASM,3,"[-1.4003353118896484, -0.7913683652877808, 1.5..."
4,Andorra,AND,4,"[-2.2527151107788086, 0.4239608645439148, 1.15..."
